In [ ]:
# prepojenie na google drive, kde si ulozime dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd  # pandas kniznica pre pracu s datami
df = pd.read_excel('/content/drive/MyDrive/uloha3_data.xlsx') # nacitanie dat do premennej df, vo forme dataframe. Moze chvilu trvat.
df  # takto sa daju data zobrazit

,case_id,t_state,p_year,p_tnum,t_manu,t_model,t_cap,t_hh,t_rd,t_ttlh,xlong,ylat
0,3086174,AK,2018,1,EWT,DW-52-900,900,50.0,52.0,75.9,-161.886581,60.780571
1,3071360,AK,2009,4,Northern Power Systems,NW100,100,32.0,21.0,42.5,-165.606064,61.524242
2,3033399,AK,2009,4,Northern Power Systems,NW100,100,32.0,21.0,42.5,-165.607483,61.524601
3,3032903,AK,2009,4,Northern Power Systems,NW100,100,37.0,21.0,47.5,-165.608871,61.525002
4,3032902,AK,2009,4,Northern Power Systems,NW100,100,37.0,21.0,47.5,-165.610184,61.525402
...,...,...,...,...,...,...,...,...,...,...,...,...
70692,3032108,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.760887,42.958794
70693,3032102,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.753693,42.946793
70694,3032124,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.771492,42.940193
70695,3032112,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.760590,42.965294


In [ ]:
df = df.dropna()  # vynechanie chybajucich hodnot
df

,case_id,t_state,p_year,p_tnum,t_manu,t_model,t_cap,t_hh,t_rd,t_ttlh,xlong,ylat
0,3086174,AK,2018,1,EWT,DW-52-900,900,50.0,52.0,75.9,-161.886581,60.780571
1,3071360,AK,2009,4,Northern Power Systems,NW100,100,32.0,21.0,42.5,-165.606064,61.524242
2,3033399,AK,2009,4,Northern Power Systems,NW100,100,32.0,21.0,42.5,-165.607483,61.524601
3,3032903,AK,2009,4,Northern Power Systems,NW100,100,37.0,21.0,47.5,-165.608871,61.525002
4,3032902,AK,2009,4,Northern Power Systems,NW100,100,37.0,21.0,47.5,-165.610184,61.525402
...,...,...,...,...,...,...,...,...,...,...,...,...
70692,3032108,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.760887,42.958794
70693,3032102,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.753693,42.946793
70694,3032124,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.771492,42.940193
70695,3032112,WY,2010,110,GE Wind,GE1.5-77,1500,80.0,77.0,118.6,-105.760590,42.965294


**Metódy strojového učenia na predikciu kapacity turbíny**



In [ ]:
# priprava dat, vsetky potrebne importy k nizsie pouzitym modelom
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# v predikcii nechceme vyuzivat case_id, takze ho vynechame
data = df.drop(columns=['case_id'])
# delime data na to co chceme predikovat (y), a to co vyuzijeme na predikciu
X = data.drop(columns=['t_cap'])
y = data['t_cap']

# Kategoricke (nie numericke) hodnoty musime skonvertovat na vektory, s tymi vedia modely ML pracovat
X = pd.get_dummies(X, columns=['t_state', 't_manu', 't_model'], drop_first=True)

# lubovolna hodnota, umozni nam dostavat pre nahodnych vyberoch stale tie iste vysledky
state = 42

# rozdelenie na testovaciu a treningovu mnozinu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)

Decision Tree MSE: 30.595827439886847
Random Forest MSE: 32.29249006364922
Neural Network MSE: 245.25208853188522


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Zopár metód, rozdelené do jednotlivých častí pre lepšiu manipuláciu. Môžete ich skúšať spúšťať s rôznymi hyperparametrami.

In [ ]:
# 1. Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=state)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

# Vyhodnotenie modelu
mse_dt = mean_squared_error(y_test, y_pred_dt)
print(f"Decision Tree MSE: {mse_dt}")

In [ ]:
# 2. Random Forest Regressor
rf_model = RandomForestRegressor(random_state=state)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest MSE: {mse_rf}")

In [ ]:
# 3. Neural Network (MLP Regressor)
# scaling - uprava dat aby boli vhodnymi vstupmi pre neuronovu siet
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nn_model = MLPRegressor(random_state=state, max_iter=150)
nn_model.fit(X_train_scaled, y_train)
y_pred_nn = nn_model.predict(X_test_scaled)

mse_nn = mean_squared_error(y_test, y_pred_nn)
print(f"Neural Network MSE: {mse_nn}")

**Metódy strojového učenia na určenie štátu, v ktorom sa turbína nachádza**

In [ ]:
# priprava dat, vsetky potrebne importy k nizsie pouzitym modelom
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# definujeme si predikovany atribut, a atributy ktore vyuzijeme na jeho predikciu
X = df.drop(columns=['case_id', 't_state'])
y = df['t_state']

# Kategoricke (nie numericke) hodnoty musime skonvertovat na vektory, s tymi vedia modely ML pracovat
X = pd.get_dummies(X, columns=['t_manu', 't_model'], drop_first=True)

# lubovolna hodnota, umozni nam dostavat pre nahodnych vyberoch stale tie iste vysledky
state = 42

# rozdelenie na testovaciu a treningovu mnozinu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)

In [ ]:
# 1. Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=state)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)

# Vyhodnotenie modelu
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Accuracy: {accuracy_dt}")

Decision Tree Accuracy: 0.9997878209208572


In [ ]:
# 2. Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=state)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")

Random Forest Accuracy: 0.9988683782445718


In [ ]:
# 3. Neural Network (MLP Classifier) - scale data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nn_clf = MLPClassifier(random_state=state, max_iter=500)
nn_clf.fit(X_train_scaled, y_train)
y_pred_nn = nn_clf.predict(X_test_scaled)

accuracy_nn = accuracy_score(y_test, y_pred_nn)
print(f"Neural Network Accuracy: {accuracy_nn}")

Neural Network Accuracy: 0.994412617582573
